In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("subhajournal/movie-rating")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Alvaro\.cache\kagglehub\datasets\subhajournal\movie-rating\versions\1


In [8]:
!pip install dash

   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB 991.0 kB/s eta 0:00:08
   --- ------------------------------------ 0.7/7.8 MB 7.1 MB/s eta 0:00:02
   ------------------------ --------------- 4.7/7.8 MB 33.8 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 45.2 MB/s eta 0:00:00


In [61]:
import pandas as pd

movies = pd.read_csv("Rotten Tomatoes Movies.csv")

# Limpieza de datos (si es necesario)
movies = movies.dropna()

In [3]:
# valores unicos de la variable "Rating"

movies["rating"].unique()

array(['PG', 'R', 'NR', 'G', 'PG-13', 'PG-13)', 'NC17', 'R)'],
      dtype=object)

In [4]:
#Imprime todas las variables

movies.columns


Index(['movie_title', 'movie_info', 'critics_consensus', 'rating', 'genre',
       'directors', 'writers', 'cast', 'in_theaters_date', 'on_streaming_date',
       'runtime_in_minutes', 'studio_name', 'tomatometer_status',
       'tomatometer_rating', 'tomatometer_count', 'audience_rating',
       'audience_count'],
      dtype='object')

In [62]:
# Crea la columna year a partir de la columna "in_theaters_date", que tiene la fecha de estreno de la película

movies["year"] = pd.to_datetime(movies["in_theaters_date"]).dt.year

# Divide los géneros en listas

movies["genre"] = movies["genre"].str.split(",")

# Manejar valores nulos reemplazándolos con listas vacías y eliminando espacios en los géneros

movies["genre"] = movies["genre"].apply(
    lambda x: [genre.strip() for genre in x] if isinstance(x, list) else []
)

# Crea una nueva columna "genre_count" que contenga el número de géneros de cada película

movies["genre_count"] = movies["genre"].apply(len)


# elimina filas donde audiencecount sea mayor a 10M

movies = movies[movies["audience_count"] <= 10000000]

# Modifica PG-13) a PG-13 y R) a R en Rating

movies["rating"] = movies["rating"].str.replace("PG-13)", "PG-13").str.replace("R)", "R")




In [43]:
import numpy as np

# haz una tabla de conteos de peliculas por studio_name

estudio_conteo = movies["studio_name"].value_counts()

# sustituye los valores de studio_name que contengan la palabra warner en mayuscula o minuscula por el valor Warner Bros, y los que contengan la palabra universal por Universal por Universal Pictures, los que contengan la palabra lion por Lionsgate, y los que contengan la palabra sony por Sony Pictures

# Sustituyendo valores según las condiciones
# Sustituyendo valores según las condiciones
movies['studio_name'] = np.where(
    movies['studio_name'].str.contains('warner', case=False, na=False),
    'Warner Bros',
    np.where(
        movies['studio_name'].str.contains('universal', case=False, na=False),
        'Universal Pictures',
        np.where(
            movies['studio_name'].str.contains('lion', case=False, na=False),
            'Lionsgate',
            np.where(
                movies['studio_name'].str.contains('sony', case=False, na=False),
                'Sony Pictures',
                np.where(
                    movies['studio_name'].str.contains('disney', case=False, na=False),
                    'Walt Disney Pictures',
                    np.where(
                        movies['studio_name'].str.contains('century fox', case=False, na=False),
                        '20th Century Fox',
                        np.where(
                            movies['studio_name'].str.contains('paramount', case=False, na=False),
                            'Paramount Pictures',
                            movies['studio_name']
                        )
                    )
                )
            )
        )
    )
)


In [40]:
# valores unicos de genre en genre_expanded

genre_expanded["genre"].unique()

array(['Action & Adventure', ' Comedy', ' Drama',
       ' Science Fiction & Fantasy', 'Comedy', 'Classics',
       ' Kids & Family', ' Classics', ' Mystery & Suspense', 'Drama',
       ' Romance', 'Art House & International', ' Faith & Spirituality',
       'Documentary', ' Special Interest', ' Horror', 'Animation',
       'Horror', ' Western', ' Sports & Fitness',
       ' Musical & Performing Arts', 'Kids & Family',
       'Mystery & Suspense', ' Art House & International', ' Television',
       'Cult Movies', ' Cult Movies', ' Animation', ' Documentary',
       'Science Fiction & Fantasy', ' Anime & Manga',
       'Musical & Performing Arts', ' Gay & Lesbian', 'Romance',
       'Western'], dtype=object)

In [92]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Expansión de géneros para análisis
genre_expanded = movies.explode("genre")

# Aplicación Dash
app = Dash(__name__)

# Estilos personalizados
custom_styles = {
    "background_color_dark_red": "#8B0000",
    "text_color": "#F8F9FA",
    "accent_color": "#000000",
    "font_size_large": "40px",
    "font_size_medium": "26px",
    "font_size_small": "26px",
    "text_black": "#000000"
}

app.layout = html.Div([
    html.H1(
        "EVOLUCIÓN DE TENDENCIAS EN EL MUNDO DEL CINE",
        style={
            "textAlign": "center",
            "marginBottom": "20px",
            "color": custom_styles["text_color"],
            "fontSize": custom_styles["font_size_large"]
        }
    ),
    html.P(
        """
        Este dashboard interactivo permite explorar la evolución de diversas métricas clave en el mundo del cine, 
        como las calificaciones de la audiencia, las valoraciones de los críticos y el número de votos recibidos 
        por las películas. Utiliza los controles a continuación para personalizar tu análisis y descubrir patrones 
        interesantes a lo largo del tiempo.
        """,
        style={
            "textAlign": "center",
            "color": custom_styles["text_color"],
            "fontSize": custom_styles["font_size_small"],
            "marginBottom": "30px"
        }
    ),
    
    html.Div([
        html.Label(
            "Selecciona la Métrica que quieres analizar:",
            style={"marginBottom": "10px", "color": custom_styles["text_color"], "fontSize": custom_styles["font_size_medium"]}
        ),
        dcc.Dropdown(
            id="metric-dropdown",
            options=[
                {"label": "Nota audiencia", "value": "audience_rating"},
                {"label": "Nota críticos", "value": "tomatometer_rating"},
                {"label": "Número de votos", "value": "audience_count"}
            ],
            value="audience_rating",
            style={
                "width": "50%",
                "marginBottom": "30px",
                "backgroundColor": custom_styles["background_color_dark_red"],
                "color": custom_styles["text_black"],
                "border": f"1px solid {custom_styles['accent_color']}",
                "fontSize": custom_styles["font_size_medium"]
            }
        ),
        
        html.Label(
            "Selecciona el Rango de Años:",
            style={"marginBottom": "10px", "marginTop": "30px", "color": custom_styles["text_color"], "fontSize": custom_styles["font_size_medium"]}
        ),
        dcc.RangeSlider(
            id="year-slider",
            min=movies["year"].min(),
            max=movies["year"].max(),
            step=1,
            marks={i: str(i) for i in range(movies["year"].min(), movies["year"].max() + 1, 5)},
            value=[movies["year"].min(), movies["year"].max()],
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ], style={"margin": "20px"}),

    html.Div([
        html.P(
            "Este primer gráfico muestra la distribución de la métrica seleccionada por clasificación de películas. Puedes analizar cómo las distintas clasificaciones tienen distribuciones de puntuaciones distintas.",
            style={"color": custom_styles["text_color"], "fontSize": custom_styles["font_size_small"]}
        ),
        dcc.Graph(id="rating-graph", style={"marginBottom": "30px"}),
        
        html.P(
            "A continuación, este gráfico de barras muestra el promedio de la métrica seleccionada para cada género de película. Permite identificar cuáles géneros tienen un mejor rendimiento en la métrica elegida, como las calificaciones de la audiencia o los votos totales. Es útil para analizar tendencias generales y comparar el desempeño promedio de distintos géneros en el periodo seleccionado.",
            style={"color": custom_styles["text_color"], "fontSize": custom_styles["font_size_small"]}
        ),
        dcc.Graph(id="genre-graph", style={"marginBottom": "30px"}),
        
        html.P(
            "El siguiente gráfico de dispersión muestra la relación entre la duración de las películas y la métrica seleccionada. Puedes identificar si hay una tendencia entre la duración y el éxito de las películas y encontrar películas concretas de un periodo concreto y con una duración que quieras con buenas valoraciones.",
            style={"color": custom_styles["text_color"], "fontSize": custom_styles["font_size_small"]}
        ),
        dcc.Graph(id="runtime-graph", style={"marginBottom": "30px"}),
        
        html.P(
            "Finalmente tenemos gráfico circular que muestra las proporciones de películas en función del género o la clasificación por edades. Este gráfico permite analizar qué tipos de películas eran más recurrentes en cada época.",
            style={"color": custom_styles["text_color"], "fontSize": custom_styles["font_size_small"]}
        ),
        html.Label(
            "Selecciona la categoría para el gráfico circular:",
            style={"marginTop": "20px", "color": custom_styles["text_color"], "fontSize": custom_styles["font_size_medium"]}
        ),
        dcc.Dropdown(
            id="pie-category-dropdown",
            options=[
                {"label": "Género", "value": "genre"},
                {"label": "Clasificación", "value": "rating"}
            ],
            value="genre",
            style={
                "width": "50%",
                "marginBottom": "20px",
                "backgroundColor": custom_styles["background_color_dark_red"],
                "color": custom_styles["text_black"],
                "border": f"1px solid {custom_styles['accent_color']}",
                "fontSize": custom_styles["font_size_medium"]
            }
        ),
        dcc.Graph(id="pie-chart")
    ], style={"margin": "20px"})
], style={"backgroundColor": custom_styles["background_color_dark_red"], "padding": "20px"})

@app.callback(
    [Output("rating-graph", "figure"),
     Output("genre-graph", "figure"),
     Output("runtime-graph", "figure"),
     Output("pie-chart", "figure")],
    [Input("metric-dropdown", "value"),
     Input("year-slider", "value"),
     Input("pie-category-dropdown", "value")]
)
def update_graphs(selected_metric, selected_years, pie_category):
    # Filtrar datos por años seleccionados
    filtered_data = movies[(movies["year"] >= selected_years[0]) & (movies["year"] <= selected_years[1])]
    genre_filtered_data = genre_expanded[(genre_expanded["year"] >= selected_years[0]) & 
                                         (genre_expanded["year"] <= selected_years[1])]
    
    # Gráfico 1: Gráfico de violín
    rating_fig = px.violin(
        filtered_data,
        x="rating",
        y=selected_metric,
        box=True,
        points="all"
    )
    
    # Gráfico 2: Barras apiladas por género y clasificación
    genre_avg = genre_filtered_data.groupby("genre")[selected_metric].mean().reset_index()
    genre_fig = px.bar(genre_avg, x="genre", y=selected_metric)
    
    # Gráfico 3: Gráfico de dispersión
    runtime_fig = px.scatter(
        filtered_data, 
        x="runtime_in_minutes", 
        y=selected_metric, 
        labels={"runtime_in_minutes": "Duración (minutos)", selected_metric: "Valor Promedio"},
        hover_data=["movie_title"]
    )
    
    # Gráfico 4: Gráfico de torta (Pie Chart)
    if pie_category == "genre":
        pie_data = genre_filtered_data["genre"].value_counts().reset_index()
        pie_data.columns = [pie_category, "count"]
    else:
        pie_data = filtered_data["rating"].value_counts().reset_index()
        pie_data.columns = [pie_category, "count"]
    
    pie_fig = px.pie(
        pie_data,
        names=pie_category,
        values="count"
    )
    
    return rating_fig, genre_fig, runtime_fig, pie_fig

if __name__ == "__main__":
    app.run_server(debug=True)